# Tune Hyperparameters of RNN and LSTM Models

Determine an optimal model type and hyperparameters for predicting NFL play calls. A hyperband search strategy is used to tune the kind of recurrent network (simple RNN or LSTM) along with a range of hyperparameters. This script is meant to run with GPU using parallel strategies on an HPC system.  

## Load Libraries

In [6]:
import json
import os
import time

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import keras_tuner as kt
from keras import metrics

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

pd.options.mode.chained_assignment = None

## Check TensorFlow Version and Find Available GPUs

In [7]:
print(f"TensorFlow Version {tf.__version__}")
print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("---- GPUs ----")
for gpu in tf.config.list_physical_devices('GPU'): print(gpu)

TensorFlow Version 2.8.0
Number of GPUs Available:  0
---- GPUs ----


## Define Functions For Model Specs and Data Processing

In [8]:
def specify(specs):
    """
    Make specifications for running the model tuning script.
    Sets directories for storing results and global variables.
    ---------------------------------------------
    Inputs: A .json file containing user specifications
    Returns: Various directories, a list of continous features, max play lags,
    and distribution strategy
    """
    # determine whether to run locally or on hpc
    HPC = specs['HPC']['value']

    # based on hpc decision, set directories for data and storing results
    # also decide distribution strategy for keras tuner parallelization 
    if HPC:
        # hpc data dir
        data_dir = os.getcwd() + '/processed_pbp.csv'

        # store results 
        results_dir = os.getcwd() + '/search_results'
        tb_dir = os.getcwd() + '/tb'

        # use mirrored strategy: one device with (potentially) multiple GPUs
        strat = tf.distribute.MirroredStrategy()
    
    else:
        # local data dir
        data_dir = specs['LOCAL_DATA_DIR']['value']

        results_dir = specs['LOCAL_RESULTS_DIR']['value']
        tb_dir = specs['LOCAL_TB_DIR']['value']

        # use one device strategy
        strat = tf.distribute.OneDeviceStrategy(device="/cpu:0")

    # get a list of continous feature variables
    cont_feats = specs['CONT_FEATS']['value']

    # get max play lags
    max_lag = specs['MAX_PLAY_LAG']['value']

    return data_dir, results_dir, tb_dir, cont_feats, strat, max_lag
# ********************************************************************
def add_lagged_play_calls(input_df, max_lag):
    """
    Given a pbp data frame, add feature columns
    for up to max_lag lagged play call values. The 
    first max_lag play calls will be dropped for each 
    team.
    ---------------------------------------------
    Inputs: input_df: Pandas df shape (total_plays, n_features)
            max_lag: (int) The maximum number of lagged play call
            values to add as columns to the pbp data frame
    Returns: A pbp data frame 
    """
    for lag in range(max_lag):
        input_df['play_lag' + str(lag + 1)] = input_df.groupby(['posteam'])['pass'].shift(lag + 1)
        
    return input_df.dropna()
# ********************************************************************
def arange_by_pos_team(input_df):
    """
    Given a pbp data frame, convert to a list
    of arrays where each array is the sequence of 
    plays for each team for the full season
    ---------------------------------------------
    Input: Pandas df shape (total_plays, n_features)
    Returns: List of arrays
    """

    # get an array of the team names 
    teams = input_df['posteam'].unique()

    # initialize a list to store arrays
    teams_ls = []

    # add the sequence of plays for each team to the list
    for i in range(len(teams)):
        pos_team = np.array(input_df[input_df['posteam'] == teams[i]])
        teams_ls.append(pos_team)
    
    return teams_ls
# ********************************************************************
def sequence_padding(ls):
    """
    Given a list of play sequence arrays, pad the 
    arrays to uniform length and combine  
    ---------------------------------------------
    Input: List of arrays
    Returns: Numpy array shape (n_teams, max_play_sequence, n_features)
    """

    # find the maximum size of the dimensions in the sequence arrays
    max_shape = [0,0]
    for a in ls:
        if max_shape[0] < a.shape[0]:
            max_shape[0] = a.shape[0]
        if max_shape[1] < a.shape[1]:
            max_shape[1] = a.shape[1]

    # append the padded sequences and combine to 3D array 
    play_seqs = []
    for a in ls:
        play_seqs.append(np.pad(a, pad_width=((0, max_shape[0] - a.shape[0]),
                                 (0, max_shape[1] - a.shape[1])), mode='constant'))
    
    play_seqs = np.stack(play_seqs)

    return play_seqs
# ********************************************************************
def process_data(input_df, seq_len, feat_list):
    """
    Converts a pbp data frame into X and y matrices 
    for training/validation. Continous features are standardized 
    and normalized.   
    ---------------------------------------------
    Inputs: input_df: Pandas df shape (total_plays, n_features)
            seq_len: (int) The number of prior plays to condsider
                     for modeling. E.g. a value of 10 means the model 
                     will look at a sequence of 10 plays and predict 
                     the 10th play

    Returns: X: np array shape ((max_play_sequence - discarded time steps) * n_samples, seq_len, n_features)
                The feature matrix
             y: np array shape ((max_play_sequence - discarded time steps) * n_samples, 1, 1)
    """
    # scale the continous features of the input
    scaler = StandardScaler()
    normalizer = Normalizer()

    input_df.loc[:,feat_list] = scaler.fit_transform(input_df.loc[:,feat_list])
    input_df.loc[:,feat_list] = normalizer.fit_transform(input_df.loc[:,feat_list])

    # get the entire sequence of plays for each team
    seqs = sequence_padding(arange_by_pos_team(input_df))
    
    # convert sequences to length 'seq_len'
    flag = 0
    for sample in range(seqs.shape[0]):
        temp = np.array([seqs[sample,i:i+seq_len,:] for i in range(seqs.shape[1] - seq_len + 1)])
        if flag==0:
            X = temp
            flag = 1
        else:
            X = np.concatenate((X,temp)) # feature matrix
    
    # output matrix (the last observation of each sequence)
    y = X[:, seq_len-1, 24][:,np.newaxis,np.newaxis]

    # remove unwanted columns
    X = np.delete(X, 24, 2)
    X = X[:, :, 4:]

    return np.asarray(X).astype(np.float32), np.asarray(y).astype(np.float32)
# ********************************************************************


## Get Specifications and Load Data Set

In [9]:
# open json file
with open('/Users/joe/documents/Masters_Project/NFL-Play-Call-Prediction-with-LSTM-Neural-Networks/src/specifications.json') as f:
    specifications = json.load(f)

# get specs
data_dir, results_dir, tb_dir, cont_feats, strat, max_lag = specify(specifications)

In [10]:
# load data
pbp = add_lagged_play_calls(pd.read_csv(data_dir), max_lag)

# split into training, validation, retraining, and test sets
train_df = pbp.iloc[0:14557,:]       # first 8 weeks of the season
val_df = pbp.iloc[14557:21467,:]     # weeks 9-12
retrain_df = pbp.iloc[0:21467,:]     # weeks 1-12 
test_df = pbp.iloc[21467:,:]         # weeks 13-17    


## Define Custom Classes for RNN/LSTM Models

Custom classes are required to tune the data preprocessing i.e. how many prior plays to consider in a sequence for the recurrent networks.

In [7]:
# rnn class
class rnn_hypermodel(kt.HyperModel):
    def build(self, hp):
        """
        Builds and compiles a recurrent neural network
        with specified hyperparameters to tune. Chooses
        between a simple RNN and an LSTM model. 
        ---------------------------------------------
        Input: hp (null) A null argument that defines the
        hyperparameter space
        Returns: A compiled recurrent neural network
        """

        # start with a sequential model
        clf = keras.Sequential()

        # tune the sequence length
        seq_len = hp.Int("seq_len", min_value = 1, max_value = 15)

        # get the size of the feature space
        num_feats = (92 + seq_len) - 5 

        # tune which kind of recurrent model to use (simple rnn or lstm)
        model_type = hp.Choice("model_type", ["rnn", "lstm"])

        if model_type == "rnn":
            # add the recurrent layer; tune units, activation function, and dropout rates
            with hp.conditional_scope("model_type", ["rnn"]):
                clf.add(keras.layers.SimpleRNN(units = hp.Int("RNNunits", min_value = 50, max_value = num_feats),
                                               input_shape = (seq_len, num_feats), 
                                               activation = hp.Choice("RNNactivation", ["tanh", "elu"]),
                                               dropout = hp.Float("RNNdropout", min_value = 0.1, max_value = 0.5),
                                               recurrent_dropout = hp.Float("RNNrec_dropout", min_value = 0.1, max_value = 0.5)))

        if model_type == "lstm":
            # add the lstm layer; certain defaults are required to run GPU implementation (activations, rec dropout, unroll, use bias)
            with hp.conditional_scope("model_type", ["lstm"]):
                clf.add(keras.layers.LSTM(units = hp.Int("LSTMunits", min_value = 50, max_value = num_feats),
                                          input_shape = (seq_len, num_feats),
                                          dropout = hp.Float("LSTMdropout", min_value = 0.1, max_value = 0.5)))
    
        # tune the number of hidden dense layers
        for i in range(hp.Int("num_dense_layers", min_value = 1, max_value = 3)):

            # add dropout, tune the rate
            clf.add(keras.layers.Dropout(rate = hp.Float("MLPdropout", min_value = 0.1, max_value = 0.5)))

            if i == 0:
                # tune size of first dense layer
                clf.add(keras.layers.Dense(units = hp.Int("MLPunits" + str(i + 1), min_value = 25, max_value = 102),
                                          activation = "elu",
                                          kernel_initializer = "he_normal"))

            if i == 1:
                # tune size of second dense layer
                clf.add(keras.layers.Dense(units = hp.Int("MLPunits" + str(i + 1), min_value = 15, max_value = 80),
                                          activation = "elu",
                                          kernel_initializer = "he_normal"))

            if i == 2:
                # tune size of second dense layer
                clf.add(keras.layers.Dense(units = hp.Int("MLPunits" + str(i + 1), min_value = 5, max_value = 60),
                                          activation = "elu",
                                          kernel_initializer = "he_normal"))
    
        # add the output layer
        clf.add(keras.layers.Dense(1, activation = "sigmoid"))

        # tune the learning rate
        lr = hp.Float("lr", min_value=1e-5, max_value = 1e-2, sampling = "log")

        # compile model
        clf.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Nadam(learning_rate=lr, clipnorm = 1.0),
                    metrics = [metrics.AUC(), metrics.Precision(), metrics.Recall()])
    
        return clf


    def fit(self, hp, model, train_df, feat_list, max_lag, validation_data = None, **kwargs):
        """
        Given a model and trial hyperparams, fit to 
        training data and evaluate on validation, if provided  
        ---------------------------------------------
        Inputs: hp: (null) A null argument that defines the
                hyperparameter space
                model: (object) a compiled neural net
                train_df: (Pandas df) input data 
                feat_list: (ls) continous features
                max_lag: (int) maximum play call lag to consider  
        Returns: History object
        """
        
        # get the number of lags used in model building to process data 
        seq_len = hp.get("seq_len")

        # get the number of lag features to remove for shape compatability
        feats_rem = max_lag - seq_len
        
        # crop training df
        cropped_train_df = train_df.iloc[:,:-feats_rem]

        # get input and output training matrices
        X_train, y_train = process_data(cropped_train_df, seq_len, feat_list)
        
        if validation_data is not None:
            # crop validation data
            cropped_val_df = validation_data.iloc[:,:-feats_rem]

            # get input and output validation matrices
            X_val, y_val = process_data(cropped_val_df, seq_len, feat_list)
            validation_data = (X_val, y_val)

        return model.fit(
            X_train,
            y_train,
            validation_data = validation_data,
            **kwargs,
        )
        

## Hyperparameter Tuning

In [8]:
# build keras tuner for rnn model
with strat.scope():
    tuner = kt.Hyperband(
        rnn_hypermodel(),
        objective = kt.Objective("val_loss", direction = "min"),
        max_epochs = 80,
        factor = 2,
        directory = results_dir,
        project_name = 'HPC_search1',
        distribution_strategy = strat,
        overwrite = True
    )

INFO:tensorflow:Reloading Oracle from existing project /Users/joe/documents/mas_proj_results/testing1/oracle.json
INFO:tensorflow:Reloading Tuner from /Users/joe/documents/mas_proj_results/testing1/tuner0.json


In [ ]:
# define callbacks 
lr_sched = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.5)
erly_stp = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
tb = tf.keras.callbacks.TensorBoard(log_dir = tb_dir)

tuner.search(
    train_df = train_df,
    feat_list = cont_feats,
    max_lag = max_lag,
    validation_data = val_df,
    epochs = 40,
    batch_size = 32,
    callbacks = [lr_sched, erly_stp, tb]
)

## Retrain the Model Using the Best Hyperparameters and Evaluate on Test Data

In [9]:
# get the best 20 models from the search
best_hps = tuner.get_best_hyperparameters(5)

# get hyper model
hyp_mod = rnn_hypermodel()

# initialize param results
param_results = pd.DataFrame()

In [10]:
for hp in best_hps:
    # build model with hyper params
    mod = hyp_mod.build(hp)

    # fit model to retrain data and evaluate on test
    hist = hyp_mod.fit(hp, mod, retrain_df, cont_feats, max_lag, validation_data = test_df, verbose = 0)
    
    # join hyperparam values with metrics for this trial
    temp_df = pd.concat([pd.DataFrame(hp.values, index = [0]), pd.DataFrame(hist.history)], axis = 1)
    
    # remove numbers at end of metrics for columns compatibility 
    temp_df.columns = temp_df.columns.str.replace('_[0-9]','', regex=True)
    temp_df.columns = temp_df.columns.str.replace('[0]','', regex=True)

    # join along rows with new trials 
    param_results = pd.concat([param_results, temp_df], axis = 0)
    

In [13]:
param_results.to_csv(results_dir + '/param_results.csv')